In [1]:
import requests, time
from bs4 import BeautifulSoup as bs, SoupStrainer as ss 
import pandas as pd, numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
prunes = ['Ancient_Greek','International_Phonetic_Alphabet','Greek_language','Help:IPA_for_Greek']
plumbs = ['Help:', 'Talk:']
loop = ['Philosophy','Education','Learning',
        'Knowledge','Awareness','Quality_(philosophy)']
loopies = [ ['Lyndon_LaRouche','LaRouche_movement'], 
           ['World_War_II','Allies_of_World_War_II','Declaration_by_United_Nations'] ]

# Harvest wiki data
There is a main central loop. We'll expand from that and collect some main highways to this page.

In [77]:
def links(page):
    '''
    Given a page, return a list of the hrefs appearing in 
    the first paragraph
    
    Returns: list(str):  
    '''
    url = 'https://en.wikipedia.org/wiki/%s' %page
    r = requests.get(url)
    main_div = ss('div', {'id': 'mw-content-text'})
    soup = bs(r.text,'lxml', parse_only=main_div)
    
    p = soup.find('p')
    links = [a['href'] for a in p.find_all('a')]
    
    refs = [link[6:] for link in links if link[:6]=='/wiki/']
    good_refs = [ref for ref in refs if ref not in prunes 
                             and ':' not in ref 
                             and 'disambiguation' not in ref]
    return good_refs

In [11]:
def all_links(page):
    '''
    Given a page, return a list of all the hrefs appearing
    on the page 
    
    Returns: list(str):  
    '''
    
    url = 'https://en.wikipedia.org/wiki/%s' %page
    r = requests.get(url)
    main_div = ss('div', {'id': 'mw-content-text'})
    soup = bs(r.text,'lxml', parse_only=main_div)
    
    ps = soup.find_all('p')
    links = [a['href'] for p in ps for a in p.find_all('a')]
    refs = [link[6:] for link in links if link[:6]=='/wiki/']
    good_refs = [ref for ref in refs if ref not in prunes 
                             and ':' not in ref 
                             and 'disambiguation' not in ref]
    return good_refs

In [102]:
foobar('Violin')

['/wiki/String_instrument',
 '/wiki/Violin_family',
 '/wiki/Violino_piccolo',
 '/wiki/Kit_violin',
 '/wiki/Strings_(music)',
 '/wiki/Perfect_fifth',
 '/wiki/Bow_(music)',
 '/wiki/Pizzicato',
 '/wiki/Western_classical_music',
 '/wiki/Folk_music',
 '/wiki/Country_music',
 '/wiki/Bluegrass_music',
 '/wiki/Jazz',
 '/wiki/Electric_violin',
 '/wiki/Rock_music',
 '/wiki/Indian_music',
 '/wiki/Iranian_music',
 '/wiki/Fiddle',
 '/wiki/Italian_Peninsula',
 '#cite_note-1',
 '#cite_note-heron-2',
 '#cite_note-3',
 '/wiki/Stradivari',
 '/wiki/Guarneri',
 '/wiki/Amati',
 '/wiki/Brescia',
 '/wiki/Cremona',
 '/wiki/Jacob_Stainer',
 '/wiki/Austria',
 '#cite_note-NYT-201430407-4',
 '#cite_note-5',
 '/wiki/Saxony',
 '/wiki/Bohemia',
 '/wiki/Mirecourt',
 '/wiki/Sears',
 '/wiki/Wood',
 '/wiki/Electric_violin',
 '/wiki/Musical_acoustics',
 '/wiki/Pickup_(music_technology)',
 '/wiki/Guitar_amplifier',
 '/wiki/Catgut',
 '/wiki/Nylon_6',
 '/wiki/Luthier',
 '#cite_note-6',
 '/wiki/Viola',
 '/wiki/Vitula',
 '#ci

In [53]:
# find title of random page
page = 'Special:Random'
url = 'https://en.wikipedia.org/wiki/%s' %page
r = requests.get(url)
title_soup = bs(r.text,'lxml')
title = title_soup.find('link', rel='canonical')['href'][30:]
print title

Brain_Drain_(comics)


In [104]:
# create dictionary!!!!
d = {}

for i in xrange(10):
    page = 'Special:Random'
    url = 'https://en.wikipedia.org/wiki/%s' %page
    r = requests.get(url)

    # find title
    title_soup = bs(r.text,'lxml')
    title = title_soup.find('link', rel='canonical')['href'][30:]

    # find all links
    main_div = ss('div', {'id': 'mw-content-text'})
    soup = bs(r.text,'lxml', parse_only=main_div)
    ps = soup.find_all('p')
    links = [a['href'] for p in ps for a in p.find_all('a')]
    refs = [link[6:] for link in links if link[:6]=='/wiki/']
    good_refs = [ref for ref in refs if ref not in prunes 
                             and ':' not in ref 
                             and 'disambiguation' not in ref]
    
    d[title] = good_refs

In [105]:
d

{'Blue_fescue': [],
 'Bohumil_Makovsky': ['Music_director',
  'Oklahoma_State_University',
  'Kappa_Kappa_Psi',
  'Fraternities_and_sororities',
  'Bohemia',
  'Czech_language',
  'Clarkson,_Nebraska',
  'Oklahoma_City,_Oklahoma',
  'Delmar_Gardens',
  'University_of_Tulsa'],
 'Claude_Mongeau': ['President',
  'Chief_Executive_Officer',
  'Canadian_National_Railway',
  'E._Hunter_Harrison',
  'Montreal',
  'Quebec',
  'Bombardier_Transportation',
  'Bell_Canada',
  'Bain_%26_Company'],
 'Cycling_at_the_2008_Summer_Paralympics_%E2%80%93_Men%27s_individual_pursuit_(B%26VI_1%E2%80%933)': ['2008_Summer_Paralympics',
  'Laoshan_Velodrome',
  'Kieran_Modra',
  'Tyson_Lawrence',
  '2004_Summer_Paralympics',
  'Robert_Crowe'],
 'Ganban%27yoku': ['Spa',
  'Thailand',
  'Granite',
  'Detoxification_(alternative_medicine)',
  'Circulatory_system',
  'Skin',
  'Spa',
  'Tokyo_Dome_City'],
 'Government_Girls%27_Postgraduate_College,_Rampur': ['Rampur,_Uttar_Pradesh',
  'M.J.P._Rohilkhand_University

In [108]:
G = nx.DiGraph()
nodes = d.keys()

for key in d.keys():
    nodes += d[key]

G.add_nodes_from(nodes)

In [110]:
edges = []

for key in d.keys():
    children = d[key]
    for child in children:
        edges.append( (key,child) )

G.add_edges_from(edges)

In [113]:
pr = nx.pagerank(G,alpha=0.8)

In [114]:
pr_list = []
for key in pr.keys():
    data_tuple = (key, pr[key])
    pr_list.append(data_tuple)

In [123]:
sorted(pr_list, key=lambda x: x[1], reverse=True)

[('Tyson_Lawrence', 0.011198926683704365),
 ('Meanings_of_minor_planet_names', 0.011198926683704365),
 ('2008_Summer_Paralympics', 0.011198926683704365),
 ('Laoshan_Velodrome', 0.011198926683704365),
 ('Kieran_Modra', 0.011198926683704365),
 ('Minor_planet', 0.011198926683704365),
 ('Minor_Planet_Center', 0.011198926683704365),
 ('Robert_Crowe', 0.011198926683704365),
 ('2004_Summer_Paralympics', 0.011198926683704365),
 ('Astronomical_naming_conventions', 0.011198926683704365),
 ('International_Astronomical_Union', 0.011198926683704365),
 ('Lutz_D._Schmadel', 0.011198926683704365),
 ('Skin', 0.011010715582746468),
 ('Spa', 0.011010715582746468),
 ('Circulatory_system', 0.011010715582746468),
 ('Detoxification_(alternative_medicine)', 0.011010715582746468),
 ('Thailand', 0.011010715582746468),
 ('Granite', 0.011010715582746468),
 ('Tokyo_Dome_City', 0.011010715582746468),
 ('American_football', 0.010869557257028045),
 ('Rock_Island_Independents', 0.010869557257028045),
 ('Monmouth_Colle

In [122]:
# sorted page rank list
[x[0] for x in sorted(pr_list, key=lambda x: x[1], reverse=True)]

['Tyson_Lawrence',
 'Meanings_of_minor_planet_names',
 '2008_Summer_Paralympics',
 'Laoshan_Velodrome',
 'Kieran_Modra',
 'Minor_planet',
 'Minor_Planet_Center',
 'Robert_Crowe',
 '2004_Summer_Paralympics',
 'Astronomical_naming_conventions',
 'International_Astronomical_Union',
 'Lutz_D._Schmadel',
 'Skin',
 'Spa',
 'Circulatory_system',
 'Detoxification_(alternative_medicine)',
 'Thailand',
 'Granite',
 'Tokyo_Dome_City',
 'American_football',
 'Rock_Island_Independents',
 'Monmouth_College',
 'Frankford_Yellow_Jackets',
 'Green_Bay_Packers',
 'National_Football_League',
 'New_York_Yankees_(NFL)',
 'Green_Bay_Packers_Hall_of_Fame',
 'University_Grants_Commission_(India)',
 'E._Hunter_Harrison',
 'Rampur,_Uttar_Pradesh',
 'Bain_%26_Company',
 'Bombardier_Transportation',
 'Nawabs_of_Rampur',
 'M.J.P._Rohilkhand_University',
 'Murtaza_Ali_Khan',
 'Raza_Library',
 'Government_of_India',
 'Quebec',
 'Canadian_National_Railway',
 'Chief_Executive_Officer',
 'Bell_Canada',
 'Geographic_coo

In [125]:
# dummy example
foo = ['amin', 'cass', 'siddharth']
bar = ['cass', 'siddharth', 'amin']

print foo.index('amin')
print bar.index('amin')

0
2


In [22]:
count = 0
for i in xrange(100):
    page = 'Special:Random'
    count +=  len(all_links(page))
print count/100.0

16.79


In [7]:
links('Mathematics')

['Quantity',
 'Number',
 'Mathematical_structure',
 'Space',
 'Calculus',
 'Definitions_of_mathematics']

In [8]:
# generate a collection of links from random pages
pages = []
for tick in xrange(10):
    page = 'Special:Random'
    pages += links(page)

In [20]:
df = pd.DataFrame(data=[[1,2,3],[4,5,6]], columns=['a','b','c'])
df

,a,b,c
0,1,2,3
1,4,5,6


In [25]:
df.iloc(2)

In [96]:
all_links('Special:Random')

['Burlington,_Vermont',
 'National_Register_of_Historic_Places',
 'Church_Street_Marketplace']

# Random walks on wiki-graph

In [91]:
# random walk on wikipedia
random_walks = []
for tick in xrange(20):
    page = 'Special:Random'
    L = links(page)
    seen = []
    while len(L) > 0 and page not in seen:
        seen.append(page)
        r = np.random.randint(0,len(L))
        page = L[r]
        L = links(page)
    seen.append(page)
    if len(L) > 0 and len(seen)>1:
        random_walks.append(seen[1:])

In [78]:
[len(x) for x in random_walks]

[8, 17, 6, 9, 17, 24, 8, 16, 3]

In [94]:
rws += random_walks

In [58]:
def loop_len(rw):
    '''
    Given a random walk, returns the length of the loop that closed it
    '''
    repeat = rw[-1]
    index = rw.index(repeat)
    return len(rw)-index

In [100]:
print sorted([(loop_len(x), len(x)) for x in rws], key=lambda x: x[0], reverse=True)

[(8, 17), (8, 12), (7, 20), (6, 10), (6, 22), (6, 9), (6, 10), (5, 22), (4, 9), (4, 29), (4, 13), (4, 8), (3, 7), (3, 12), (3, 18), (3, 49), (3, 7), (3, 3), (3, 33), (3, 5), (3, 20), (3, 8), (3, 17), (3, 6), (3, 24), (3, 8), (3, 16), (3, 3), (3, 3), (3, 60), (3, 8), (3, 14), (3, 30), (3, 15), (3, 12)]


### Random walk graph

In [96]:
G_rw = nx.DiGraph()
for rw in rws:
    for s, t in zip(rw, rw[1:]):
        if (s,t) not in G_rw.edges():
            G_rw.add_edge(s, t, weight=0)
        G_rw[s][t]['weight'] += 1
    s = rw[-1]
    t = rw[0]
    if (s,t) not in G_rw.edges():
        G_rw.add_edge(s,t, weight=0)
    G_rw[s][t]['weight'] += 1

In [97]:
len(G_rw.edges())

531

In [98]:
# save the graph
nx.write_graphml(G_rw, 'random_walk.graphml')

# First link graph

In [164]:
for tick in xrange(2):
    path = []
    page = 'Special:Random'
    count = 0
    while page not in loop and count < 1000:
        L = links(page)
        if len(L)>0:
            page = L[0]
            path.append(page)
            count += 1
            if count > 20:
                print page
        else:
            path = []
            page = 'Special:Random'
            count = 0
    paths.append(path)

In [165]:
print [len(path) for path in paths]

[8, 9, 5, 5, 7, 9, 6, 11, 14, 7, 5, 11, 6, 9, 18, 13, 9, 9, 4, 8, 17, 13, 6, 14, 6, 12, 11, 6, 16, 12, 11, 7, 6, 14, 7, 12, 11, 17, 15]


Longest paths to date:
- Cavalry has length 18  
- Professional_baseball has length 16

## Three node counter example

In [80]:
G = nx.DiGraph()

In [82]:
nodes = [1,2,3]
G.add_nodes_from(nodes)

In [83]:
edges = [(1,2),(1,3),(2,1)]
G.add_edges_from(edges)

In [88]:
G[1][3]['weight'] = 5

In [92]:
nx.pagerank(G,alpha=0.99)

{1: 0.35325071354186066, 2: 0.20680209157046414, 3: 0.43994719488767464}

# Building the graph

In [202]:
G = nx.DiGraph()

# add the loop
for s, t in zip(loop, loop[1:]):
    if (s,t) not in G.edges():
        G.add_edge(s,t)
    G.add_edge(loop[-1],loop[0])

# add the rest of the data
for path in paths:
    for s, t in zip(path, path[1:]):
        if (s,t) not in G.edges():
            G.add_edge(s,t)

In [203]:
nx.write_graphml(G, 'wiki_graph.graphml')

In [204]:
pr = nx.pagerank(G)

In [207]:
ranks = sorted([(page, pr[page]) for page in pr.keys()], key=lambda x:x[1], reverse=True)

In [208]:
ranks

[('Knowledge', 0.0757529652755453),
 ('Awareness', 0.06523532268274902),
 ('Philosophy', 0.06114870545410897),
 ('Quality_(philosophy)', 0.05630535474794927),
 ('Education', 0.05456416348648078),
 ('Learning', 0.04717971395119743),
 ('Science', 0.036629327326936384),
 ('Natural_science', 0.017118819106942407),
 ('Quantity', 0.01635446461758999),
 ('Europe', 0.016114062625987365),
 ('Property_(philosophy)', 0.014716512316255839),
 ('Continent', 0.01451217062339361),
 ('Mathematics', 0.014327611482862924),
 ('Landmass', 0.013150562421188915),
 ('Earth', 0.011993195449314927),
 ('Geography', 0.011955810614303949),
 ('Geopolitics', 0.011009433523222034),
 ('Biology', 0.01062801921189196),
 ('British_English', 0.009851308291259768),
 ('United_Kingdom', 0.00918882943887515),
 ('Communication', 0.007449889384977922),
 ('Meaning_(semiotics)', 0.007147623368535581),
 ('Semiotics', 0.006890697254559592),
 ('Sport', 0.0066768393342391315),
 ('Ferdinand_de_Saussure', 0.00667231005768),
 ('Linguist